# MCP Server Toolset Implementation

This notebook creates a custom **Model Context Protocol (MCP) toolset** that exposes a `shakeIde` command to MCP clients.

The command shakes your JetBrains IDE window using IntelliJ Platform SDK.

In [ ]:
%use intellij-platform

Load the MCP Server bundled plugin using its `com.intellij.mcpServer` ID.

In [ ]:
loadPlugins("com.intellij.mcpServer")

Implement the `McpToolset` extension with an example code that shakes the current IDE using a built-it `DialogEarthquakeShaker` utility class.
Using the `registerExtension` helper, register the implemented extension as `com.intellij.mcpServer.mcpToolset`.

In [ ]:
import com.intellij.openapi.application.ApplicationManager
import com.intellij.openapi.ui.DialogEarthquakeShaker
import com.intellij.openapi.wm.WindowManager
import com.intellij.mcpserver.McpToolset
import com.intellij.mcpserver.annotations.McpDescription
import com.intellij.mcpserver.annotations.McpTool
import com.intellij.mcpserver.project
import kotlinx.coroutines.delay
import kotlin.coroutines.coroutineContext

class ShakeMcpToolset : McpToolset {

    @McpTool
    @McpDescription("Shakes your JetBrains IDE 😵‍💫")
    suspend fun shakeIde(
        @McpDescription("How many times to shake your IDE?")
        times: Int = 1
    ) {
        val project = coroutineContext.project
        val window = WindowManager.getInstance().getFrame(project)

        repeat(times) {
            DialogEarthquakeShaker.shake(window)
            delay(500)
        }
    }
}

registerExtension(McpToolset.EP, ShakeMcpToolset())

## Testing

Make sure that the JetBrains MCP Server is enabled in [⚙️ Settings | Tools | MCP Server](jetbrains://idea/settings?name=Tools--MCP+Server) and copy its URL, like `http://127.0.0.1:64342/sse`

As soon as the extension is registered, you can test it using the [MCP Inspector tool](https://modelcontextprotocol.io/docs/tools/inspector):

```shell
npx @modelcontextprotocol/inspector
```

When running, _🔗 Open inspector with token pre-filled:_ `http://localhost:6274/?MCP_PROXY_AUTH_TOKEN=foo`

Configure the connection and set the _Transport Type_ to `SSE`, the MCP Server URL, and establish the connection.

Find the `shakeIde` entry in the tool list, set the `times` parameter to any number, and click the _Run tool_ button.

## Integration with standalone plugins

To register the extension within a standalone plugin, use:
```xml
<idea-plugin>
  ...
  <extensions defaultExtensionNs="com.intellij">
    <mcpServer.mcpToolset implementation="com.example.ShakeMcpToolset"/>
  </extensions>
</idea-plugin>
```